![](images/EscUpmPolit_p.gif "UPM")

## Load and clean

In [1]:
# General import and load data
import pandas as pd
import numpy as np

from pandas import Series, DataFrame

# Training and test spliting
from sklearn.cross_validation import train_test_split
from sklearn import preprocessing

# Estimators
from sklearn.svm import SVC

# Evaluation
from sklearn import metrics
from sklearn.cross_validation import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Optimization
from sklearn.grid_search import GridSearchCV

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)


# if matplotlib is not set inline, you will not see plots
#alternatives auto gtk gtk2 inline osx qt qt5 wx tk
#%matplotlib auto
#%matplotlib qt
%matplotlib inline
%run plot_learning_curve

/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/anaconda2/lib/python2.7/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
ERROR:root:File `u'plot_learning_curve.py'` not found.


In [2]:
df = pd.read_csv('./data-churn/train.csv')
df.loc[df["Partner"] == "Yes", "Partner"] = 1
df.loc[df["Partner"] == "No", "Partner"] = 0
df.loc[df["Dependents"] == "Yes", "Dependents"] = 1
df.loc[df["Dependents"] == "No", "Dependents"] = 0
df.loc[df["PhoneService"] == "Yes", "PhoneService"] = 1
df.loc[df["PhoneService"] == "No", "PhoneService"] = 0
df.loc[df["InternetService"] == "No", "InternetService"] = 0
df.loc[df["InternetService"] == "DSL", "InternetService"] = 1
df.loc[df["InternetService"] == "Fiber optic", "InternetService"] = 2

df.loc[df["OnlineSecurity"] == "No", "OnlineSecurity"] = 0
df.loc[df["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 0
df.loc[df["OnlineSecurity"] == "Yes", "OnlineSecurity"] = 1
df.loc[df["TechSupport"] == "No", "TechSupport"] = 0
df.loc[df["TechSupport"] == "No internet service", "TechSupport"] = 0
df.loc[df["TechSupport"] == "Yes", "TechSupport"] = 1

df.loc[df["StreamingTV"] == "No", "StreamingTV"] = 0
df.loc[df["StreamingTV"] == "No internet service", "StreamingTV"] = 0
df.loc[df["StreamingTV"] == "Yes", "StreamingTV"] = 1

df.loc[df["StreamingMovies"] == "No", "StreamingMovies"] = 0
df.loc[df["StreamingMovies"] == "No internet service", "StreamingMovies"] = 0
df.loc[df["StreamingMovies"] == "Yes", "StreamingMovies"] = 1

df.loc[df["Contract"] == "Two year", "Contract"] = 0
df.loc[df["Contract"] == "One year", "Contract"] = 1
df.loc[df["Contract"] == "Month-to-month", "Contract"] = 2
df.loc[df["DeviceProtection"] == "No", "DeviceProtection"] = 0
df.loc[df["DeviceProtection"] == "Yes", "DeviceProtection"] = 1
df.loc[df["DeviceProtection"] == "No internet service", "DeviceProtection"] = 2

df.loc[df["gender"] == "Male", "gender"] = 0
df.loc[df["gender"] == "Female", "gender"] = 1

df.loc[df["PaymentMethod"] == "Bank transfer (automatic)", "PaymentMethod"] = 0
df.loc[df["PaymentMethod"] == "Mailed check", "PaymentMethod"] = 1
df.loc[df["PaymentMethod"] == "Electronic check", "PaymentMethod"] = 1
df.loc[df["PaperlessBilling"] == "Yes", "PaperlessBilling"] = 1
df.loc[df["PaperlessBilling"] == "No", "PaperlessBilling"] = 0

df.loc[df["MultipleLines"] == "No", "MultipleLines"] = 0
df.loc[df["MultipleLines"] == "No phone service", "MultipleLines"] = 0
df.loc[df["MultipleLines"] == "Yes", "MultipleLines"] = 1

df.drop(['OnlineBackup'], axis=1, inplace=True)
df.drop(['PaymentMethod'], axis=1, inplace=True)
df.drop(['TotalCharges'], axis=1, inplace=True)

len(df)

5543

In [3]:
#Check types are numeric
df.dtypes

customerID            int64
gender                int64
SeniorCitizen         int64
Partner               int64
Dependents            int64
tenure                int64
PhoneService          int64
MultipleLines         int64
InternetService       int64
OnlineSecurity        int64
DeviceProtection      int64
TechSupport           int64
StreamingTV           int64
StreamingMovies       int64
Contract              int64
PaperlessBilling      int64
MonthlyCharges      float64
Churn                 int64
dtype: object

In [4]:
#Check there are not missing values
df.isnull().any()

customerID          False
gender              False
SeniorCitizen       False
Partner             False
Dependents          False
tenure              False
PhoneService        False
MultipleLines       False
InternetService     False
OnlineSecurity      False
DeviceProtection    False
TechSupport         False
StreamingTV         False
StreamingMovies     False
Contract            False
PaperlessBilling    False
MonthlyCharges      False
Churn               False
dtype: bool

# Train and test splitting

We use the same techniques we applied in the Iris dataset. 

Nevertheless, we need to remove the column 'Survived' 

In [5]:
# Features of the model
features = ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 
            'InternetService','OnlineSecurity','DeviceProtection','TechSupport','StreamingTV','StreamingMovies',
            'Contract','PaperlessBilling','MonthlyCharges']
# Transform dataframe in numpy arrays
X = df[features].values
y = df['Churn'].values
len(y)

5543

# Define model

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
types_of_kernels = ['linear', 'rbf', 'poly']

kernel = types_of_kernels[0]
gamma = 3.0

# Create kNN model
model = SVC(kernel=kernel, probability=True, gamma=gamma)
model.fit(X_train, y_train)

predicted = model.predict(X_test)
expected = y_test
metrics.accuracy_score(expected, predicted)

0.8073593073593074

Ok, we get around 80% of accuracy!

In [7]:
dfTest = pd.read_csv('./data-churn/test_nolabel.csv')

In [8]:
dfTest.loc[dfTest["Partner"] == "Yes", "Partner"] = 1
dfTest.loc[dfTest["Partner"] == "No", "Partner"] = 0
dfTest.loc[dfTest["Dependents"] == "Yes", "Dependents"] = 1
dfTest.loc[dfTest["Dependents"] == "No", "Dependents"] = 0
dfTest.loc[dfTest["PhoneService"] == "Yes", "PhoneService"] = 1
dfTest.loc[dfTest["PhoneService"] == "No", "PhoneService"] = 0
dfTest.loc[dfTest["InternetService"] == "No", "InternetService"] = 0
dfTest.loc[dfTest["InternetService"] == "DSL", "InternetService"] = 1
dfTest.loc[dfTest["InternetService"] == "Fiber optic", "InternetService"] = 2
dfTest.loc[dfTest["OnlineSecurity"] == "No", "OnlineSecurity"] = 0
dfTest.loc[dfTest["OnlineSecurity"] == "No internet service", "OnlineSecurity"] = 0
dfTest.loc[dfTest["OnlineSecurity"] == "Yes", "OnlineSecurity"] = 1
dfTest.loc[dfTest["TechSupport"] == "No", "TechSupport"] = 0
dfTest.loc[dfTest["TechSupport"] == "No internet service", "TechSupport"] = 0
dfTest.loc[dfTest["TechSupport"] == "Yes", "TechSupport"] = 1

dfTest.loc[dfTest["StreamingTV"] == "No", "StreamingTV"] = 0
dfTest.loc[dfTest["StreamingTV"] == "No internet service", "StreamingTV"] = 0
dfTest.loc[dfTest["StreamingTV"] == "Yes", "StreamingTV"] = 1

dfTest.loc[dfTest["StreamingMovies"] == "No", "StreamingMovies"] = 0
dfTest.loc[dfTest["StreamingMovies"] == "No internet service", "StreamingMovies"] = 0
dfTest.loc[dfTest["StreamingMovies"] == "Yes", "StreamingMovies"] = 1

dfTest.loc[dfTest["Contract"] == "Two year", "Contract"] = 0
dfTest.loc[dfTest["Contract"] == "One year", "Contract"] = 1
dfTest.loc[dfTest["Contract"] == "Month-to-month", "Contract"] = 2
dfTest.loc[dfTest["DeviceProtection"] == "No", "DeviceProtection"] = 0
dfTest.loc[dfTest["DeviceProtection"] == "Yes", "DeviceProtection"] = 1
dfTest.loc[dfTest["DeviceProtection"] == "No internet service", "DeviceProtection"] = 2

dfTest.loc[dfTest["gender"] == "Male", "gender"] = 0
dfTest.loc[dfTest["gender"] == "Female", "gender"] = 1

dfTest.loc[dfTest["PaymentMethod"] == "Bank transfer (automatic)", "PaymentMethod"] = 0
dfTest.loc[dfTest["PaymentMethod"] == "Mailed check", "PaymentMethod"] = 1
dfTest.loc[dfTest["PaymentMethod"] == "Electronic check", "PaymentMethod"] = 1
dfTest.loc[dfTest["PaperlessBilling"] == "Yes", "PaperlessBilling"] = 1
dfTest.loc[dfTest["PaperlessBilling"] == "No", "PaperlessBilling"] = 0

dfTest.loc[dfTest["MultipleLines"] == "No", "MultipleLines"] = 0
dfTest.loc[dfTest["MultipleLines"] == "No phone service", "MultipleLines"] = 0
dfTest.loc[dfTest["MultipleLines"] == "Yes", "MultipleLines"] = 1
dfTest.drop(['OnlineBackup'], axis=1, inplace=True)
dfTest.drop(['PaymentMethod'], axis=1, inplace=True)
dfTest.drop(['TotalCharges'], axis=1, inplace=True) 
dfTest.drop(['customerID'], axis=1, inplace=True)

X2 = dfTest
X2[:]

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,MonthlyCharges
0,0,1,1,0,65,1,1,2,1,0,0,1,1,1,1,100.75
1,1,0,1,1,67,1,1,2,0,1,1,1,1,1,1,112.35
2,0,0,0,0,2,0,0,1,0,1,0,1,1,2,1,49.25
3,0,0,0,0,7,0,0,1,0,0,0,0,0,1,1,24.35
4,0,0,0,0,12,1,0,2,0,0,0,1,0,2,1,81.45
5,1,1,0,0,57,1,1,2,1,1,1,0,1,0,0,101.30
6,1,0,0,0,52,1,0,1,0,1,0,0,1,2,1,59.45
7,0,0,0,0,15,1,0,0,0,2,0,0,0,1,0,19.80
8,1,0,1,0,42,1,1,1,0,0,0,0,0,2,1,50.25
9,1,0,0,0,2,0,0,1,0,0,0,1,1,2,0,51.40


In [9]:
model = SVC(kernel=kernel, probability=True, gamma=gamma)
model.fit(X, y)
churn = model.predict(X2)
dataframeResult = pd.read_csv('./data-churn/test_nolabel.csv').filter(['customerID'], axis=1)
dataframeResult['Churn'] = churn
dataframeResult[:]

,customerID,Churn
0,21135993,0
1,13130474,0
2,7533169,1
3,2114231208,0
4,3524728,1
5,192125049,0
6,172428986,0
7,21329444,0
8,232256498,0
9,162223094,0


In [10]:
dataframeResult.to_csv('outSVM.csv', sep=',',index=False)